# Setup

In [1]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.mavasbel.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.20.2"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.mavasbel.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("CQLENG_ALLOW_SCHEMA_MANAGEMENT", "True")

'True'

# Session creation

In [3]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_nodes = [
    DefaultEndPoint(CASSANDRA_NODE_HOSTNAMES[j], CASSANDRA_NODE_RPC_PORTS[j])
    for j in range(CASSANDRA_TOTAL_NODES)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)

auth_provider = PlainTextAuthProvider(
    username=CASSANDRA_INIT_USER, password=CASSANDRA_INIT_PASSWORD
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

🔗 Connecting to: ['cassandra-node-1.mavasbel.vpn.itam.mx:9041', 'cassandra-node-2.mavasbel.vpn.itam.mx:9042', 'cassandra-node-3.mavasbel.vpn.itam.mx:9043']
✅ Connected to cluster: cassandra-cluster.mavasbel.vpn.itam.mx
🌐 Nodes found: 3


In [4]:
import pprint
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.options.display.html.use_mathjax = True
pd.options.display.html.border = 1
pd.options.display.html.table_schema = False
pd.options.display.expand_frame_repr = True

display(
    pd.DataFrame(
        [cassandra_host.__dict__ for cassandra_host in cluster.metadata.all_hosts()]
    )
    .sort_index()
    .transpose()
)

,0,1,2
endpoint,cassandra-node-2.mavasbel.vpn.itam.mx:9042,10.15.20.2:9041,10.15.20.2:9043
conviction_policy,<cassandra.policies.SimpleConvictionPolicy object at 0x000001FED1709A90>,<cassandra.policies.SimpleConvictionPolicy object at 0x000001FED371D810>,<cassandra.policies.SimpleConvictionPolicy object at 0x000001FED371DBA0>
host_id,4f1a33f3-5359-4367-9073-3507a5e80be8,e113825e-cadb-43a2-bdec-793fd0cd67ee,93a96850-1842-45fc-8543-0cddae606d2d
_datacenter,dc1,dc1,dc1
_rack,rack1,rack1,rack1
lock,<unlocked _thread.RLock object owner=0 count=0 at 0x000001FED1555E00>,<unlocked _thread.RLock object owner=0 count=0 at 0x000001FED154AF00>,<unlocked _thread.RLock object owner=0 count=0 at 0x000001FED1580640>
is_up,True,True,True
listen_address,172.19.0.3,NaN,NaN
listen_port,7002.0,NaN,NaN
broadcast_address,10.15.20.2,10.15.20.2,10.15.20.2


In [5]:
# from cassandra.cluster import ResultSet

# peers_v2: ResultSet = session.execute("SELECT * FROM system.peers_v2")
# locals: ResultSet = session.execute("SELECT * FROM system.local")

# for peer_v2 in peers_v2:
#     print(" | ".join([f"{col}: {peer_v2[col]}" for col in peers_v2.column_names]))
# print("=" * 100)
# for local in locals:
#     print(" | ".join([f"{col}: {local[col]}" for col in locals.column_names]))
# print("=" * 100)

### Keyspace

In [6]:
keyspace_name = "generic_analytics"
if CASSANDRA_START_FROM_SCRATCH:
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_name}")

In [7]:
keyspace_name = "generic_analytics"
session.execute(
    f"""
CREATE KEYSPACE IF NOT EXISTS {keyspace_name} 
WITH replication = {{
    'class': 'NetworkTopologyStrategy', 
    'dc1': {CASSANDRA_TOTAL_NODES}
}}
"""
)
session.set_keyspace(keyspace_name)

# Create Table
# Partition Key: city (distributes data)
# Clustering Column: user_id (sorts data within city)
session.execute(
    """
CREATE TABLE IF NOT EXISTS user_metrics (
    city text,
    user_id uuid,
    username text,
    session_duration int,
    last_access timestamp,
    PRIMARY KEY (city, user_id)
);
"""
)

### Insert

In [8]:
from faker import Faker

fake = Faker()

In [9]:
import uuid
from datetime import datetime
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

# 1. Prepare your statement outside the loop
query = """
INSERT INTO user_metrics (city, user_id, username, session_duration, last_access)
VALUES (?, ?, ?, ?, ?)
"""
prepared = session.prepare(query)
# prepared.consistency_level = ConsistencyLevel.QUORUM
# statement = SimpleStatement(query, consistency_level=ConsistencyLevel.QUORUM)

# 2. Create the Batch object
# LOGGED ensures atomicity but adds disk overhead, UNLOGGED is faster
batch_records = 200
batch = BatchStatement(batch_type=BatchType.LOGGED)
batch.consistency_level = ConsistencyLevel.QUORUM
print(f"Preparing batch of {batch_records} records...")
for _ in range(batch_records):
    batch.add(
        prepared,
        (
            fake.city(),
            uuid.uuid4(),
            fake.user_name(),
            fake.random_int(min=1, max=3600),
            datetime.now(),
        ),
    )

# 3. Execute the entire batch at once
session.execute(batch)
print("Batch successfully committed to the cluster.")

Preparing batch of 200 records...
Batch successfully committed to the cluster.


### Query

In [10]:
from cassandra.cluster import ResultSet
from typing import cast

print(cast(ResultSet, session.execute("SELECT count(*) FROM user_metrics")).one())

{'count': 200}


In [11]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT * FROM user_metrics LIMIT 100")
for row in rows.current_rows:
    print(row)

{'city': 'Port April', 'user_id': UUID('604c0445-7ebc-4a9f-81ac-2e71e4a5b6ed'), 'last_access': datetime.datetime(2026, 1, 11, 20, 18, 46, 313000), 'session_duration': 1255, 'username': 'jamesbrown'}
{'city': 'Port April', 'user_id': UUID('fae0786d-9607-4c5d-9da8-620905b6c08d'), 'last_access': datetime.datetime(2026, 1, 11, 20, 18, 46, 300000), 'session_duration': 1914, 'username': 'kathleenanderson'}
{'city': 'New Kimberlyton', 'user_id': UUID('38638974-692d-44cc-889a-51be2cab9b9f'), 'last_access': datetime.datetime(2026, 1, 11, 20, 18, 46, 314000), 'session_duration': 489, 'username': 'bmiller'}
{'city': 'North Tyler', 'user_id': UUID('d2f2c1b1-cdd5-4eb0-9616-b3b62d3bcbc6'), 'last_access': datetime.datetime(2026, 1, 11, 20, 18, 46, 308000), 'session_duration': 3348, 'username': 'lowerykimberly'}
{'city': 'Lake Victoriashire', 'user_id': UUID('26210da9-3ec1-4f11-941b-c947caa07ac0'), 'last_access': datetime.datetime(2026, 1, 11, 20, 18, 46, 311000), 'session_duration': 2632, 'username':

In [12]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT DISTINCT city FROM user_metrics", [])
for row in rows.current_rows:
    print(row)

{'city': 'Port April'}
{'city': 'New Kimberlyton'}
{'city': 'North Tyler'}
{'city': 'Lake Victoriashire'}
{'city': 'Connietown'}
{'city': 'Franklinberg'}
{'city': 'Port Michaelshire'}
{'city': 'Davidsonborough'}
{'city': 'Ginatown'}
{'city': 'Thorntonport'}
{'city': 'Johnsonhaven'}
{'city': 'Johnborough'}
{'city': 'Shieldsville'}
{'city': 'Snyderburgh'}
{'city': 'Smithberg'}
{'city': 'Port Michelle'}
{'city': 'Port Sarah'}
{'city': 'Deanstad'}
{'city': 'New Jeffrey'}
{'city': 'West Catherine'}
{'city': 'West Michellehaven'}
{'city': 'West Trevorfort'}
{'city': 'Arroyofurt'}
{'city': 'Pottsmouth'}
{'city': 'Marshallville'}
{'city': 'East Douglaston'}
{'city': 'Port Cynthia'}
{'city': 'Cherylshire'}
{'city': 'Hardinshire'}
{'city': 'East Brianafort'}
{'city': 'Christineberg'}
{'city': 'New Joshuahaven'}
{'city': 'Elizabethberg'}
{'city': 'East Thomas'}
{'city': 'South Christina'}
{'city': 'North Zacharyland'}
{'city': 'North Bradley'}
{'city': 'South Gregoryfurt'}
{'city': 'East Robert'}

### Find nodes storing random data

In [13]:
import random
import pprint

random_token = random.randint(-9223372036854775808, 9223372036854775807)
query = "SELECT * FROM user_metrics WHERE token(city) >= %s LIMIT 1"
row: dict = cast(ResultSet, session.execute(query, [random_token])).one()
print(f"Random user_metric: {pprint.pformat(row)}")

prepared = session.prepare("SELECT * FROM user_metrics WHERE city=? AND user_id=?")
bound = prepared.bind([row['city'], row['user_id']])
routing_key = bound.routing_key
nodes = cluster.metadata.get_replicas(keyspace_name, routing_key)

print(f"Nodes storing '{row['city']}':")
for node in nodes:
    print(f" - Host: {node.address}, Gossip Port: {node.broadcast_port}")

Random user_metric: {'city': 'Johnborough',
 'last_access': datetime.datetime(2026, 1, 11, 20, 18, 46, 305000),
 'session_duration': 2884,
 'user_id': UUID('f20b6a7d-f201-4506-8c79-f84bb6e91525'),
 'username': 'hcollier'}
Nodes storing 'Johnborough':
 - Host: 10.15.20.2, Gossip Port: 7001
 - Host: cassandra-node-2.mavasbel.vpn.itam.mx, Gossip Port: 7002
 - Host: 10.15.20.2, Gossip Port: 7003


### ORM-like

In [14]:
from typing import cast
from cassandra.cqlengine import columns
from cassandra.cqlengine.query import ModelQuerySet
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table, create_keyspace_network_topology
from cassandra.cqlengine import connection

# 1. Connect the engine
connection.set_session(session)


# 2. Define your "Generic" Model
class UserMetrics(Model):
    __table_name__ = "user_metrics"

    # FIRST primary_key=True is the Partition Key
    # city = columns.Text(primary_key=True)
    city = columns.Text(primary_key=True, partition_key=True)

    # SECOND primary_key=True is the Clustering Key
    user_id = columns.UUID(primary_key=True, default=uuid.uuid4)

    # Attributes (Data)
    username = columns.Text(index=True)
    session_duration = columns.Integer()
    last_access = columns.DateTime()


# 3. Create Keyspace and Sync Table (Equivalent to CREATE TABLE)
create_keyspace_network_topology(keyspace_name, {"dc1": CASSANDRA_TOTAL_NODES})
sync_table(UserMetrics)

In [15]:
# 4. Use it like an ORM
# Create UserMetrics
new_metric: UserMetrics = UserMetrics.create(
    city=fake.city(),
    username=fake.name(),
    session_duration=120,
    last_access=datetime.now(),
)
new_metric.save()
print(f"Saved user_metris: {new_metric}")


# 5. Query UserMetrics
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    print(f"Query user_metris: {user_metric}")

Saved user_metris: UserMetrics <city=East Brittney, user_id=7a8c8381-b2e3-41c4-8db7-638f765ae8a5>
Query user_metris: UserMetrics <city=Port April, user_id=604c0445-7ebc-4a9f-81ac-2e71e4a5b6ed>
Query user_metris: UserMetrics <city=Port April, user_id=fae0786d-9607-4c5d-9da8-620905b6c08d>
Query user_metris: UserMetrics <city=New Kimberlyton, user_id=38638974-692d-44cc-889a-51be2cab9b9f>
Query user_metris: UserMetrics <city=North Tyler, user_id=d2f2c1b1-cdd5-4eb0-9616-b3b62d3bcbc6>
Query user_metris: UserMetrics <city=Lake Victoriashire, user_id=26210da9-3ec1-4f11-941b-c947caa07ac0>
Query user_metris: UserMetrics <city=Connietown, user_id=25015173-fceb-42d5-9622-7dad007dd50e>
Query user_metris: UserMetrics <city=Franklinberg, user_id=2c1b6347-6f36-430c-ba74-bb2aefabaafb>
Query user_metris: UserMetrics <city=Port Michaelshire, user_id=f3719cc6-b069-4cd4-a1bb-70cf1d970afb>
Query user_metris: UserMetrics <city=Ginatown, user_id=642604ce-1f2f-4c27-946c-78999fce63ea>
Query user_metris: UserMet

### Insert/Create

In [16]:
from typing import cast

cast(ModelQuerySet, UserMetrics.ttl(86400)).create(
    city=fake.city(),
    username=fake.user_name(),
    session_duration=fake.random_int(10, 1000),
    last_access=fake.date_time(),
)

UserMetrics(city='Port Ashleyshire', user_id=UUID('87f75e96-d108-46d8-ace1-9ac400aa8159'), username='zpeterson', session_duration=573, last_access=datetime.datetime(2019, 4, 11, 22, 11, 53))

In [17]:
from typing import cast
from cassandra.cqlengine.query import BatchQuery
from cassandra.cqlengine.query import ModelQuerySet

with BatchQuery() as b:
    for _ in range(100):
        cast(ModelQuerySet, UserMetrics.batch(b)).create(
            city=fake.city(),
            username=fake.user_name(),
            session_duration=fake.random_int(10, 1000),
            last_access=fake.date_time(),
        )

### Delete

In [18]:
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    user_metric.delete()
    print(f"Delete user_metris: {user_metric}")

Delete user_metris: UserMetrics <city=Port April, user_id=604c0445-7ebc-4a9f-81ac-2e71e4a5b6ed>
Delete user_metris: UserMetrics <city=Port April, user_id=fae0786d-9607-4c5d-9da8-620905b6c08d>
Delete user_metris: UserMetrics <city=New Kimberlyton, user_id=38638974-692d-44cc-889a-51be2cab9b9f>
Delete user_metris: UserMetrics <city=East Diana, user_id=b4c1a064-e729-4879-bd73-91db7ef5e881>
Delete user_metris: UserMetrics <city=North Tyler, user_id=d2f2c1b1-cdd5-4eb0-9616-b3b62d3bcbc6>
Delete user_metris: UserMetrics <city=Lake Victoriashire, user_id=26210da9-3ec1-4f11-941b-c947caa07ac0>
Delete user_metris: UserMetrics <city=Connietown, user_id=25015173-fceb-42d5-9622-7dad007dd50e>
Delete user_metris: UserMetrics <city=Franklinberg, user_id=2c1b6347-6f36-430c-ba74-bb2aefabaafb>
Delete user_metris: UserMetrics <city=Port Michaelshire, user_id=f3719cc6-b069-4cd4-a1bb-70cf1d970afb>
Delete user_metris: UserMetrics <city=Ginatown, user_id=642604ce-1f2f-4c27-946c-78999fce63ea>
Delete user_metris:

In [19]:
# from cassandra.cqlengine import connection

# # This isn't a compaction, but it forces RAM data to disk so the background compaction process can see it.
# connection.execute(f"ALTER TABLE {keyspace_name}.user_metrics WITH gc_grace_seconds = 10")